In [18]:
import pandas as pd

# Load the dataset
sleep_data = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\capstone\\Fitbit Experiment.csv")

# Define a mapping of month names to their respective numbers
month_mapping = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
    'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Example: Assume 'Date' column has dates in the format 'dd-MMM'
if 'Day' in sleep_data.columns:
    sleep_data['Day'] = sleep_data['Day'].replace(month_mapping, regex=True)

# Now you can proceed with further cleaning steps
numeric_columns = ['Calories Burned', 'Steps', 'Distance', 'Minutes Sedentary', 
                   'Minutes Lightly Active', 'Minutes Fairly Active', 'Minutes Very Active',
                   'Activity Calories']

# Remove commas and convert to numeric, setting errors to NaN
for col in numeric_columns:
    sleep_data[col] = pd.to_numeric(sleep_data[col].replace({',': ''}, regex=True), errors='coerce')

# Drop rows where any of the specified columns have NaN values
sleep_data_cleaned = sleep_data.dropna(subset=numeric_columns)

# Fill remaining missing values with the median
sleep_data_cleaned.fillna(sleep_data_cleaned.median(numeric_only=True), inplace=True)

# Save the cleaned data
sleep_data_cleaned.to_csv('cleaned_sleep_data.csv', index=False)

# Verify the changes
print(sleep_data_cleaned.info())
print(sleep_data_cleaned.head())


<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 0 to 27
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Day                     28 non-null     object 
 1   Calories Burned         28 non-null     float64
 2   Steps                   28 non-null     float64
 3   Distance                28 non-null     float64
 4   Minutes Sedentary       28 non-null     float64
 5   Minutes Lightly Active  28 non-null     float64
 6   Minutes Fairly Active   28 non-null     float64
 7   Minutes Very Active     28 non-null     float64
 8   Activity Calories       28 non-null     float64
 9   Minutes Asleep          28 non-null     float64
 10  Minutes Awake           28 non-null     float64
 11  Number of Awakenings    28 non-null     float64
 12  Time in Bed             28 non-null     float64
 13  Minutes REM Sleep       28 non-null     float64
 14  Minutes Light Sleep     28 non-null     float64
 

C:\Users\sidbh\AppData\Local\Temp\ipykernel_12776\1682868199.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_data_cleaned.fillna(sleep_data_cleaned.median(numeric_only=True), inplace=True)


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

# Load the cleaned dataset
sleep_data = pd.read_csv('cleaned_sleep_data.csv')

if 'Day' in sleep_data.columns:
    sleep_data = sleep_data.drop(columns=['Day'])

# Features and target variables
X = sleep_data.drop(columns=['Minutes Asleep', 'Minutes REM Sleep', 'Minutes Deep Sleep'])  # Features
y_total_sleep = sleep_data['Minutes Asleep']  # Target variable for total sleep
y_rem_sleep = sleep_data['Minutes REM Sleep']  # Target variable for REM sleep
y_deep_sleep = sleep_data['Minutes Deep Sleep']  # Target variable for Deep sleep

# Split the data into training and testing sets
X_train, X_test, y_total_train, y_total_test, y_rem_train, y_rem_test, y_deep_train, y_deep_test = train_test_split(
    X, y_total_sleep, y_rem_sleep, y_deep_sleep, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Custom loss function to enforce sleep guidelines
def custom_loss(y_true, y_pred):
    total_sleep_pred = y_pred[:, 0]
    rem_sleep_pred = y_pred[:, 1]
    deep_sleep_pred = y_pred[:, 2]

    # Mean Squared Error for total sleep
    total_sleep_loss = K.mean(K.square(y_true[:, 0] - total_sleep_pred), axis=-1)

    # Penalty if REM sleep is less than 120 minutes
    rem_penalty = K.maximum(0.0, 120 - rem_sleep_pred)

    # Penalty if deep sleep is less than 13% or more than 23% of total sleep
    deep_sleep_penalty = K.maximum(0.0, 0.13 * total_sleep_pred - deep_sleep_pred) + \
                         K.maximum(0.0, deep_sleep_pred - 0.23 * total_sleep_pred)

    # Penalty if total sleep is outside the 6-9 hours range (360-540 minutes)
    lower_bound_penalty = K.maximum(0.0, 360 - total_sleep_pred)
    upper_bound_penalty = K.maximum(0.0, total_sleep_pred - 540)

    # Combine all penalties and the original loss
    return total_sleep_loss + rem_penalty + deep_sleep_penalty + lower_bound_penalty + upper_bound_penalty

# Build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3)  # Output layer for total sleep, REM sleep, and Deep sleep
])

# Compile the model using the custom loss function
model.compile(optimizer='adam', loss=custom_loss)

# Prepare the training data for the combined model
y_train = pd.concat([y_total_train, y_rem_train, y_deep_train], axis=1)
y_test = pd.concat([y_total_test, y_rem_test, y_deep_test], axis=1)

# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')

# Predict total sleep, REM sleep, and Deep sleep duration
predictions = model.predict(X_test)
print(f'Predicted total sleep, REM sleep, and Deep sleep for first 5 examples:\n{predictions[:5]}')


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 138212.2656 - val_loss: 161522.8438
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 138185.4375 - val_loss: 161508.5156
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 138159.1875 - val_loss: 161494.2812
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 138132.8750 - val_loss: 161477.2188
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 138106.2812 - val_loss: 161456.5781
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 138079.1562 - val_loss: 161433.4688
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 138051.7031 - val_loss: 161412.5938
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 138024.5469 - val_loss: 161392.3125
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 137997.3750 - val_loss: 161371.5312
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 137970.7188 - val_loss: 161351.3594
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 13

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
import numpy as np

# Load the cleaned dataset
sleep_data = pd.read_csv('cleaned_sleep_data.csv')

# Drop the 'Day' column
sleep_data = sleep_data.drop(columns=['Day'])

# Features and target variables
X = sleep_data.drop(columns=['Minutes Asleep', 'Minutes REM Sleep', 'Minutes Deep Sleep'])  # Features
y_total_sleep = sleep_data['Minutes Asleep']  # Target variable for total sleep
y_rem_sleep = sleep_data['Minutes REM Sleep']  # Target variable for REM sleep
y_deep_sleep = sleep_data['Minutes Deep Sleep']  # Target variable for Deep sleep

# Split the data into training and testing sets
X_train, X_test, y_total_train, y_total_test, y_rem_train, y_rem_test, y_deep_train, y_deep_test = train_test_split(
    X, y_total_sleep, y_rem_sleep, y_deep_sleep, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Custom loss function to enforce sleep guidelines and penalize negative predictions
def custom_loss(y_true, y_pred):
    total_sleep_pred = y_pred[:, 0]
    rem_sleep_pred = y_pred[:, 1]
    deep_sleep_pred = y_pred[:, 2]

    # Mean Squared Error for total sleep
    total_sleep_loss = K.mean(K.square(y_true[:, 0] - total_sleep_pred), axis=-1)

    # Penalty if REM sleep is less than 120 minutes
    rem_penalty = K.maximum(0.0, 120 - rem_sleep_pred)

    # Penalty if deep sleep is less than 13% or more than 23% of total sleep
    deep_sleep_penalty = K.maximum(0.0, 0.13 * total_sleep_pred - deep_sleep_pred) + \
                         K.maximum(0.0, deep_sleep_pred - 0.23 * total_sleep_pred)

    # Penalty if total sleep is outside the 6-9 hours range (360-540 minutes)
    lower_bound_penalty = K.maximum(0.0, 360 - total_sleep_pred)
    upper_bound_penalty = K.maximum(0.0, total_sleep_pred - 540)

    # Penalty for negative values
    negative_penalty = K.sum(K.maximum(0.0, -y_pred))

    # Combine all penalties and the original loss
    return total_sleep_loss + rem_penalty + deep_sleep_penalty + lower_bound_penalty + upper_bound_penalty + negative_penalty

# Build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3, activation='relu')  # Enforce non-negative outputs
])

# Compile the model using the custom loss function
model.compile(optimizer='adam', loss=custom_loss)

# Prepare the training data for the combined model
y_train = pd.concat([y_total_train, y_rem_train, y_deep_train], axis=1)
y_test = pd.concat([y_total_test, y_rem_test, y_deep_test], axis=1)

# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')

# Predict total sleep, REM sleep, and Deep sleep duration
predictions = model.predict(X_test)

# Clip negative predictions to 0
predictions = np.clip(predictions, 0, None)

print(f'Predicted total sleep, REM sleep, and Deep sleep for first 5 examples:\n{predictions[:5]}')


Epoch 1/100


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 138172.2656 - val_loss: 161506.0156
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 138155.3125 - val_loss: 161493.5156
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 138135.9531 - val_loss: 161480.1406
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 138114.5312 - val_loss: 161465.3438
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 138091.9844 - val_loss: 161448.5469
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 138067.6719 - val_loss: 161422.7812
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 138043.2656 - val_loss: 161390.7969
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 138017.2656 - val_loss: 161356.3594
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 137989.3281 - val_loss: 161318.5469
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 137961.1562 - val_loss: 161279.6250
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 137930.2812 - 